In [1]:
import os
import pandas as pd

### Sensor data preprocessing

1. load all bs and tx data and concatenate them to creata combined data frame seperately 
2. take the average of each measurement ie 5 measurements in each five minutesd
3. merge the bs and tx combined data frame based on date_time column. Remane all columns based on tx  or bs prefix.

In [2]:

def process_files(directory, prefix, output_filename):
    files = [f for f in os.listdir(directory) if f.startswith(prefix)]
    if not files:
        print(f"No files found with prefix '{prefix}' in '{directory}'.")
        return
    
    combined_df = pd.DataFrame()
    
    for file in files:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        
        if combined_df.empty:
            combined_df = df
        else:
            combined_df = pd.concat([combined_df, df[1:]])  # Skip header for subsequent files

    combined_df['date_time'] = pd.to_datetime(combined_df['date_time']).dt.floor('S')
    combined_df.sort_values(by='date_time', inplace=True)
    combined_df.to_csv(output_filename, index=False)
    print(f"Processed file saved as '{output_filename}'.")

if __name__ == "__main__":
    directory = './'  # Replace with your directory
    process_files(directory, 'Sensor_bs', 'sensor_bs_combined.csv')
    process_files(directory, 'Sensor_tx', 'sensor_tx_combined.csv')


/tmp/ipykernel_18767/2145707433.py:18: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  combined_df['date_time'] = pd.to_datetime(combined_df['date_time']).dt.floor('S')


Processed file saved as 'sensor_bs_combined.csv'.


/tmp/ipykernel_18767/2145707433.py:18: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  combined_df['date_time'] = pd.to_datetime(combined_df['date_time']).dt.floor('S')


Processed file saved as 'sensor_tx_combined.csv'.


In [3]:

# Load the combined CSV files
sensor_bs_path = './sensor_bs_combined.csv'
sensor_tx_path = './sensor_tx_combined.csv'

sensor_bs_df = pd.read_csv(sensor_bs_path)
sensor_tx_df = pd.read_csv(sensor_tx_path)

# Rename columns by adding prefixes
sensor_bs_df = sensor_bs_df.rename(columns=lambda x: 'bs_' + x)
sensor_tx_df = sensor_tx_df.rename(columns=lambda x: 'tx_' + x)

# Rename the date_time column back to the original for merging
sensor_bs_df = sensor_bs_df.rename(columns={'bs_date_time': 'date_time'})
sensor_tx_df = sensor_tx_df.rename(columns={'tx_date_time': 'date_time'})

# Perform an inner merge on the date_time column
merged_df = pd.merge(sensor_bs_df, sensor_tx_df, on='date_time', how='inner')

# Save the merged DataFrame to a new CSV file
merged_output_path = './sensor_merged.csv'
merged_df.to_csv(merged_output_path, index=False)
merged_df


,date_time,bs_410nm,bs_435nm,bs_460nm,bs_485nm,bs_510nm,bs_535nm,bs_560nm,bs_585nm,bs_610nm,...,tx_610nm,tx_645nm,tx_680nm,tx_705nm,tx_730nm,tx_760nm,tx_810nm,tx_860nm,tx_900nm,tx_940nm
0,2024-07-08 14:08:44,105.28,269.44,959.28,184.75,354.07,586.04,124.41,158.24,946.58,...,4786.92,590.71,361.79,784.04,165.62,206.97,495.32,483.87,214.77,109.34
1,2024-07-08 14:08:48,104.50,270.43,960.20,185.58,352.57,586.73,124.41,158.24,945.51,...,4752.99,593.58,361.79,784.04,165.62,206.13,494.43,482.70,214.77,109.34
2,2024-07-08 14:08:51,103.71,270.43,965.71,186.40,354.07,587.43,124.41,158.24,947.65,...,4783.53,593.58,367.18,784.95,166.39,207.80,495.32,483.87,214.77,110.29
3,2024-07-08 14:08:55,103.71,269.44,962.96,185.58,352.57,584.64,124.84,159.05,946.58,...,4732.64,594.06,361.79,782.23,164.86,205.30,490.89,479.20,213.44,109.34
4,2024-07-08 14:13:58,104.50,264.51,910.58,176.51,346.58,580.47,122.27,155.82,933.71,...,4651.22,591.18,353.18,775.90,164.10,209.47,499.75,503.69,218.08,109.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23763,2024-07-26 15:00:51,66.00,148.05,427.27,106.40,197.62,327.82,105.60,138.86,741.83,...,6058.00,623.28,398.40,845.61,213.71,221.99,689.37,733.38,262.49,122.65
23764,2024-07-26 15:00:55,64.43,149.03,449.32,109.70,200.61,329.21,106.46,140.07,750.40,...,6024.08,628.07,404.86,849.23,212.18,216.98,684.05,712.39,261.83,122.65
23765,2024-07-26 15:00:58,64.43,148.05,451.16,109.70,200.61,327.82,106.46,140.48,751.47,...,6154.13,629.51,408.09,866.43,214.47,219.49,687.59,712.39,258.52,123.60
23766,2024-07-26 15:01:02,64.43,150.02,451.16,111.35,200.61,330.60,106.03,139.67,747.19,...,6105.50,623.28,403.79,860.55,213.71,216.98,681.39,711.22,257.85,124.55


In [5]:
# Load the uploaded CSV files
sensor_merged_path = './sensor_merged.csv'
sensor_tx_combined_path = './sensor_tx_combined.csv'

sensor_merged_df = pd.read_csv(sensor_merged_path)
sensor_tx_combined_df = pd.read_csv(sensor_tx_combined_path)

# Extract the date_time columns
merged_dates = set(sensor_merged_df['date_time'])
tx_dates = set(sensor_tx_combined_df['date_time'])

# Find dates present in tx_dates but not in merged_dates
unique_tx_dates = tx_dates - merged_dates

# Convert the result to a DataFrame for better readability
unique_tx_dates_df = pd.DataFrame(unique_tx_dates, columns=['unique_date_time'])



unique_tx_dates_df


,unique_date_time
0,2024-07-22 10:56:52
1,2024-07-26 06:16:21
2,2024-07-15 11:16:07
3,2024-07-24 03:50:22
4,2024-07-22 09:58:42
...,...
403,2024-07-10 10:58:43
404,2024-07-25 08:27:26
405,2024-07-08 19:21:34
406,2024-07-24 00:44:57


In [7]:
# Load the uploaded CSV files again to ensure the latest data is used
sensor_bs_combined_path = './sensor_bs_combined.csv'
sensor_tx_combined_path = './sensor_tx_combined.csv'

sensor_bs_df = pd.read_csv(sensor_bs_combined_path)
sensor_tx_df = pd.read_csv(sensor_tx_combined_path)

# Remove the second part from date_time column, keeping only hour and minute along with date
sensor_bs_df['date_time'] = pd.to_datetime(sensor_bs_df['date_time']).dt.strftime('%Y-%m-%d %H:%M')
sensor_tx_df['date_time'] = pd.to_datetime(sensor_tx_df['date_time']).dt.strftime('%Y-%m-%d %H:%M')

# Group by the new date_time and take the average for rows with the same date and time value
sensor_bs_avg = sensor_bs_df.groupby('date_time').mean().reset_index()
sensor_tx_avg = sensor_tx_df.groupby('date_time').mean().reset_index()

# Rename columns by adding prefixes
sensor_bs_avg = sensor_bs_avg.rename(columns=lambda x: 'bs_' + x if x != 'date_time' else x)
sensor_tx_avg = sensor_tx_avg.rename(columns=lambda x: 'tx_' + x if x != 'date_time' else x)

# Perform an inner merge on the date_time column
merged_avg_df = pd.merge(sensor_bs_avg, sensor_tx_avg, on='date_time', how='inner')

# Save the merged DataFrame to a new CSV file
merged_avg_output_path = './sensor_avg_merged.csv'
merged_avg_df.to_csv(merged_avg_output_path, index=False)

In [8]:
sensor_bs_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6029 entries, 0 to 6028
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  6029 non-null   object 
 1   bs_410nm   6029 non-null   float64
 2   bs_435nm   6029 non-null   float64
 3   bs_460nm   6029 non-null   float64
 4   bs_485nm   6029 non-null   float64
 5   bs_510nm   6029 non-null   float64
 6   bs_535nm   6029 non-null   float64
 7   bs_560nm   6029 non-null   float64
 8   bs_585nm   6029 non-null   float64
 9   bs_610nm   6029 non-null   float64
 10  bs_645nm   6029 non-null   float64
 11  bs_680nm   6029 non-null   float64
 12  bs_705nm   6029 non-null   float64
 13  bs_730nm   6029 non-null   float64
 14  bs_760nm   6029 non-null   float64
 15  bs_810nm   6029 non-null   float64
 16  bs_860nm   6029 non-null   float64
 17  bs_900nm   6029 non-null   float64
 18  bs_940nm   6029 non-null   float64
dtypes: float64(18), object(1)
memory usage: 895.1+ K

In [9]:
sensor_tx_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6030 entries, 0 to 6029
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  6030 non-null   object 
 1   tx_410nm   6030 non-null   float64
 2   tx_435nm   6030 non-null   float64
 3   tx_460nm   6030 non-null   float64
 4   tx_485nm   6030 non-null   float64
 5   tx_510nm   6030 non-null   float64
 6   tx_535nm   6030 non-null   float64
 7   tx_560nm   6030 non-null   float64
 8   tx_585nm   6030 non-null   float64
 9   tx_610nm   6030 non-null   float64
 10  tx_645nm   6030 non-null   float64
 11  tx_680nm   6030 non-null   float64
 12  tx_705nm   6030 non-null   float64
 13  tx_730nm   6030 non-null   float64
 14  tx_760nm   6030 non-null   float64
 15  tx_810nm   6030 non-null   float64
 16  tx_860nm   6030 non-null   float64
 17  tx_900nm   6030 non-null   float64
 18  tx_940nm   6030 non-null   float64
dtypes: float64(18), object(1)
memory usage: 895.2+ K

In [10]:
merged_avg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6028 entries, 0 to 6027
Data columns (total 37 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  6028 non-null   object 
 1   bs_410nm   6028 non-null   float64
 2   bs_435nm   6028 non-null   float64
 3   bs_460nm   6028 non-null   float64
 4   bs_485nm   6028 non-null   float64
 5   bs_510nm   6028 non-null   float64
 6   bs_535nm   6028 non-null   float64
 7   bs_560nm   6028 non-null   float64
 8   bs_585nm   6028 non-null   float64
 9   bs_610nm   6028 non-null   float64
 10  bs_645nm   6028 non-null   float64
 11  bs_680nm   6028 non-null   float64
 12  bs_705nm   6028 non-null   float64
 13  bs_730nm   6028 non-null   float64
 14  bs_760nm   6028 non-null   float64
 15  bs_810nm   6028 non-null   float64
 16  bs_860nm   6028 non-null   float64
 17  bs_900nm   6028 non-null   float64
 18  bs_940nm   6028 non-null   float64
 19  tx_410nm   6028 non-null   float64
 20  tx_435nm